### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 


In [1]:
import numpy as np
import pandas as pd
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [2]:
data.head()

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2


#### (A)

Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

In [3]:
from __future__ import division

In [4]:
print 'P(Red|Stolen):',float(3/5)

print 'P(SUV|Stolen):', float(2/5)

print 'P(Domestic|Stolen):', float(2/5)

print 'P(Red|Not Stolen):', float(2/5)

print 'P(SUV|Not Stolen):', float(3/5)

print 'P(Domestic|Not Stolen):', float(3/5)

P(Red|Stolen): 0.6
P(SUV|Stolen): 0.4
P(Domestic|Stolen): 0.4
P(Red|Not Stolen): 0.4
P(SUV|Not Stolen): 0.6
P(Domestic|Not Stolen): 0.6


#### (B)

Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

$$
P(Stolen|\{Red,Domestic\})\sim P(Red|Stolen)P(Domestic|Stolen)P(!SUV|Stolen)P(Stolen)=3/5*2/5*(1-3/5)*1/2=0.048
$$


$$
P(Not Stolen|\{Red,Domestic\})\sim P(Red|Not Stolen)P(Domestic|Not Stolen)P(!SUV|Non Stolen)P(Non Stolen)=2/5*3/5*(1-2/5)*1/2=0.072
$$

$$
P(Stolen|\{Red,Domestic\}) < P(Not Stolen|\{Red,Domestic\}) = Not Stolen
$$

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:


### (A)

Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

In [5]:
data_train=pd.read_csv("dia_train.csv", usecols=range(1,10)) 
#y_train=data_train.iloc[:,1] 
#X_train=data_train.iloc[:,2:] 

def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [6]:
data_train.head()

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
0,0,3,158,64,13,387,31.2,0.295,24
1,0,0,84,64,22,66,35.8,0.545,21
2,0,9,120,72,22,56,20.8,0.733,48
3,0,4,110,76,20,100,28.4,0.118,27
4,1,2,100,66,20,90,32.9,0.867,28


In [7]:
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

In [8]:
#now train the classifier based on the training sample
dp=trainNaiveBayes(data_train)

In [9]:
dp

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.6991525,2.690909,111.4667,69.20606,27.2,127.0061,31.70909,0.4686848,28.39394
sigma1,NaN,2.610061,24.69189,11.71329,10.43692,91.48614,6.337613,0.2917503,8.537362
mu2,0.3008475,4.070423,144.1408,74.56338,33.47887,209.2113,35.22394,0.6390423,35.78873
sigma2,NaN,3.51862,30.62646,13.79931,9.762697,126.921,6.258491,0.439042,10.26355


### (B)

Perform the classification for the test sample. 

In [10]:
from scipy import stats
data_test=pd.read_csv("dia_test.csv", usecols=range(1,10))
y_test=data_test.iloc[:,0]
X_test=data_test.iloc[:,1:]

C=classifyNaiveBayes(data_test,dp)

### (C) 

Compare your result to y_test and report the classification accuracy.

In [11]:
#classification accuracy (over test set)
print "The classification accuracy over the test set is:", 1.0*sum(C==y_test)/len(C)

The classification accuracy over the test set is: 0.791139240506


### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.
 

In [12]:
data_train=pd.read_csv("EM_train.csv", usecols=range(1,5))
y_Label_train=data_train.iloc[:,0] 
X_Label_train=data_train.iloc[:,1:] 

data_test=pd.read_csv("EM_test.csv", usecols=range(1,5))
y_Label_test=data_test.iloc[:,0]
X_Label_test=data_test.iloc[:,1:]

data_Unlabel=pd.read_csv("EM_Unlabel.csv", usecols=range(1,4))
X_Unlabel=data_Unlabel.iloc[:,0:]

In [13]:
def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

In [14]:
def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

### (A) 

Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

In [15]:
dp=trainNaiveBayesDiscrete(data_train)
dp

[           y    0    1    2
 1  0.5555556    0  0.2    0
 2        NaN  0.2    0  0.2
 3        NaN  0.2    0    0
 4        NaN  0.2  0.2  0.2
 5        NaN    0    0    0
 6        NaN  0.4  0.6  0.6,            y     0     1     2
 1  0.4444444   0.5  0.25  0.25
 2        NaN  0.25   0.5   0.5
 3        NaN     0  0.25     0
 4        NaN     0     0     0
 5        NaN  0.25     0     0
 6        NaN     0     0  0.25]

In [16]:
C=classifyNaiveBayesDiscrete(data_test,dp)

In [17]:
acc=(100.0*sum(C==y_Label_test)/len(y_Label_test))
print "We correctly classified {0} percent of the artificial data set based on the labeled data only".format(acc)

We correctly classified 63.8888888889 percent of the artificial data set based on the labeled data only


### (B) 

Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook).

In [18]:
from numpy import linalg as LA
import math 

In [19]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label_test)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if ((sum(np.sum(dp1[0]-dp[0])**2)) + sum(np.sum(dp1[1]-dp[1])**2))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp


In [20]:
#perform EM estimation for theta
dpEM=EM(X_Label_test,y_Label_test,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(data_test,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_Label_test)/len(y_Label_test)
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -744.261718032
After EM we correctly classified 91.6666666667 percents of the trips


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 



In [21]:
data=pd.read_csv("EM_Cluster.csv")
X=data.iloc[:,1:]

In [22]:
X.head()

,0,1,2
0,5,6,6
1,4,6,6
2,1,5,5
3,1,3,4
4,5,6,2


In [23]:
np.random.seed(2016)
#initialize theta randomly
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b


In [24]:
dp

[           y         0         1         2
 1  0.8967054  0.203711  0.233433  0.052900
 2        NaN  0.218506  0.246143  0.221832
 3        NaN  0.206894  0.205944  0.216376
 4        NaN  0.128907  0.191068  0.025485
 5        NaN  0.179253  0.079939  0.346481
 6        NaN  0.062729  0.043473  0.136927,
            y         0         1         2
 1  0.1032946  0.275895  0.145768  0.248003
 2        NaN  0.275194  0.118057  0.187649
 3        NaN  0.020212  0.159788  0.189117
 4        NaN  0.103416  0.346520  0.101663
 5        NaN  0.109967  0.161480  0.147634
 6        NaN  0.215316  0.068387  0.125933]

In [25]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM2(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label_test)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if ((sum(np.sum(dp1[0]-dp[0])**2)) + sum(np.sum(dp1[1]-dp[1])**2))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        #if haslabels:    
         # for i in X_Label.index:
        #  yi=y_Label[i]
        #     P=dp[yi][cols[0]][1];
         #   for j in X_Label.columns:
        #      P=P*dp[yi][j][X_Label[j][i]]
           # 3L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp


In [26]:
#perform EM estimation for theta
dpEM=EM2([],[],X,dp)
#OS test
C=classifyNaiveBayesDiscrete(data_test,dpEM) #classify test data with a new theta given by EM
print dpEM

Iteration 1: log maximum liklihood = -550.945248537
[           y           0          1           2
1  0.8261141   0.2225422   0.154627   0.1671593
2        NaN  0.09910279  0.2622571   0.2167209
3        NaN   0.1703821  0.1170647   0.1504559
4        NaN   0.2322533  0.1461682  0.07273329
5        NaN   0.1964081  0.1359577   0.2219638
6        NaN  0.07931145  0.1839254   0.1709668,            y           0           1           2
1  0.1738859   0.2572156  0.08693984   0.3012523
2        NaN   0.1864182   0.1233049  0.06579016
3        NaN  0.01208871   0.1010832  0.05198585
4        NaN   0.1563089   0.1271265    0.311697
5        NaN   0.1075238   0.2851768  0.09565213
6        NaN   0.2804448   0.2763688   0.1736226]


### (A) 

Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 


In [27]:
C

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    1
34    1
35    0
Name: y, dtype: int64

### (B) 

Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

In [28]:
np.random.seed(2015)
#initialize theta randomly

test = []
values = []
for q in range(10):
    dp[0].iloc[0,0]=np.random.uniform(0,1)
    dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
    for j in range(1,len(dp[0].T)):
        b=np.random.uniform(0,1,len(dp[0]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[0].iloc[:,j]=b
        test.append(dp[0].iloc[:,j])
    for j in range(1,len(dp[1].T)):
        b=np.random.uniform(0,1,len(dp[1]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[1].iloc[:,j]=b
        test.append(dp[1].iloc[:,j])
        #perform EM estimation for theta
    dpEM=EM2([],[],X,dp)
    C=classifyNaiveBayesDiscrete(data_test,dpEM) #classify test data with a new theta given by EM
    values.append(C)
    

Iteration 1: log maximum liklihood = -534.408980849
Iteration 1: log maximum liklihood = -549.039120484
Iteration 1: log maximum liklihood = -556.261873821
Iteration 1: log maximum liklihood = -552.530038876
Iteration 1: log maximum liklihood = -546.241774499
Iteration 1: log maximum liklihood = -550.025433635
Iteration 1: log maximum liklihood = -553.888122335
Iteration 1: log maximum liklihood = -553.888164557
Iteration 1: log maximum liklihood = -556.0418992


In [29]:
df = pd.DataFrame(values)

In [30]:
#the columns are the observations and the rows are the different iterations with a new theta
#the values are the different classifications 
df

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
y,1,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,1
y,1,0,1,0,0,1,1,1,1,0,...,0,1,0,1,0,1,1,0,0,0
y,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,1
y,1,1,1,1,0,1,1,1,1,0,...,0,1,1,1,0,1,1,1,1,1
y,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
y,0,0,0,0,1,0,0,0,1,1,...,0,0,0,0,1,1,0,1,0,0
y,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
y,1,1,0,1,1,1,1,0,1,1,...,1,0,0,1,1,1,1,1,1,1
y,0,0,1,0,1,0,0,1,1,1,...,0,1,0,0,1,1,1,1,0,0
y,0,0,1,0,0,1,1,1,0,0,...,0,1,0,1,0,0,0,0,0,0


In [31]:
#the average label value
df.mean()

0     0.5
1     0.3
2     0.4
3     0.3
4     0.3
5     0.5
6     0.5
7     0.4
8     0.5
9     0.3
10    0.3
11    0.4
12    0.4
13    0.5
14    0.0
15    0.5
16    0.4
17    0.3
18    0.4
19    0.3
20    0.3
21    0.4
22    0.3
23    0.5
24    0.5
25    0.3
26    0.3
27    0.4
28    0.4
29    0.5
30    0.3
31    0.5
32    0.5
33    0.5
34    0.4
35    0.4
dtype: float64

In [32]:
#the standard error for each value
stats.sem(df)

array([ 0.16666667,  0.15275252,  0.16329932,  0.15275252,  0.15275252,
        0.16666667,  0.16666667,  0.16329932,  0.16666667,  0.15275252,
        0.15275252,  0.16329932,  0.16329932,  0.16666667,  0.        ,
        0.16666667,  0.16329932,  0.15275252,  0.16329932,  0.15275252,
        0.15275252,  0.16329932,  0.15275252,  0.16666667,  0.16666667,
        0.15275252,  0.15275252,  0.16329932,  0.16329932,  0.16666667,
        0.15275252,  0.16666667,  0.16666667,  0.16666667,  0.16329932,
        0.16329932])